In [1]:
import flwr

flwr.__version__

'1.17.0'

In [2]:
from pathlib import Path

SYFT_FLWR_PROJECT_PATH = Path("./fl-diabetes-prediction")
assert SYFT_FLWR_PROJECT_PATH.exists()

In [12]:
!flwr run {SYFT_FLWR_PROJECT_PATH}

Loading project configuration... 
Success
2025-05-12 15:59:39.805 | INFO     | fl_diabetes_prediction.task:<module>:37 - Using device: mps
/Users/khoaguin/Desktop/projects/OpenMined/test/fl4health_demo/fl-diabetes-prediction/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:918: PydanticDeprecatedSince20: `__get_validators__` is deprecated and will be removed, use `__get_pydantic_core_schema__` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/Users/khoaguin/Desktop/projects/OpenMined/test/fl4health_demo/fl-diabetes-prediction/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, Deprecati

In [5]:
import syft_flwr

DS = "khoa@openmined.org"
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"
DO3 = "do3@openmined.org"
DO4 = "do4@openmined.org"
DO5 = "do5@openmined.org"

syft_flwr.bootstrap(SYFT_FLWR_PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2, DO3, DO4, DO5])

/Users/khoaguin/Desktop/projects/OpenMined/test/fl4health_demo/fl-diabetes-prediction/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:918: PydanticDeprecatedSince20: `__get_validators__` is deprecated and will be removed, use `__get_pydantic_core_schema__` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/Users/khoaguin/Desktop/projects/OpenMined/test/fl4health_demo/fl-diabetes-prediction/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


FileExistsError: File 'fl-diabetes-prediction/main.py' already exists

In [8]:
PARTITION_PATH = Path("./dataset/splits")
MOCK_PATHS = [PARTITION_PATH / f"client_{i}" / "mock" for i in range(5)]
MOCK_PATHS

[PosixPath('dataset/splits/client_0/mock'),
 PosixPath('dataset/splits/client_1/mock'),
 PosixPath('dataset/splits/client_2/mock'),
 PosixPath('dataset/splits/client_3/mock'),
 PosixPath('dataset/splits/client_4/mock')]

In [10]:
syft_flwr.run(SYFT_FLWR_PROJECT_PATH, MOCK_PATHS)

2025-05-12 15:57:54.797 | INFO     | syft_rds.orchestra:remove_rds_stack_dir:92 - Skipping removal, as path /var/folders/lm/3l67gzrs2ws7vfw7djym8klw0000gn/T/fl-diabetes-prediction_26c3e2c7-43a0-44ac-969f-31640f31e578 does not exist
2025-05-12 15:57:54.801 | INFO     | syft_rds.orchestra:setup_rds_server:219 - Launching mock RDS server in /private/var/folders/lm/3l67gzrs2ws7vfw7djym8klw0000gn/T/fl-diabetes-prediction_26c3e2c7-43a0-44ac-969f-31640f31e578
2025-05-12 15:57:54.821 | INFO     | syft_rds.orchestra:setup_rds_server:219 - Launching mock RDS server in /private/var/folders/lm/3l67gzrs2ws7vfw7djym8klw0000gn/T/fl-diabetes-prediction_26c3e2c7-43a0-44ac-969f-31640f31e578
2025-05-12 15:57:54.839 | INFO     | syft_rds.orchestra:setup_rds_server:219 - Launching mock RDS server in /private/var/folders/lm/3l67gzrs2ws7vfw7djym8klw0000gn/T/fl-diabetes-prediction_26c3e2c7-43a0-44ac-969f-31640f31e578
2025-05-12 15:57:54.856 | INFO     | syft_rds.orchestra:setup_rds_server:219 - Launching mock

2025-05-12 15:57:54.932 | INFO     | syft_flwr.run_simulation:_run_simulated_flwr_project:80 - 📝 Log directory: /Users/khoaguin/Desktop/projects/OpenMined/test/fl4health_demo/fl-diabetes-prediction/fl-diabetes-prediction/simulation_logs
2025-05-12 15:57:54.932 | INFO     | syft_flwr.run_simulation:_run_simulated_flwr_project:84 - Running DS client 'khoa@openmined.org' with config path /var/folders/lm/3l67gzrs2ws7vfw7djym8klw0000gn/T/fl-diabetes-prediction_26c3e2c7-43a0-44ac-969f-31640f31e578/khoa@openmined.org.config.json
2025-05-12 15:57:54.932 | INFO     | syft_flwr.run_simulation:_run_simulated_flwr_project:99 - Running DO client 'do1@openmined.org' with config path /var/folders/lm/3l67gzrs2ws7vfw7djym8klw0000gn/T/fl-diabetes-prediction_26c3e2c7-43a0-44ac-969f-31640f31e578/do1@openmined.org.config.json on mock dataset /Users/khoaguin/Desktop/projects/OpenMined/test/fl4health_demo/fl-diabetes-prediction/dataset/splits/client_0/mock
2025-05-12 15:57:54.932 | INFO     | syft_flwr.run_s